<a href="https://colab.research.google.com/github/joao-dias-25/dataeng-spark/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: hh24miss)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# Setting up PySpark

In [1]:
%pip install pyspark

In [31]:

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests

class ETLFlow:
    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def extract_from_file(self, format: str, path: str, **kwargs) -> DataFrame:
        df = self.spark.read.format(format).load(path)
        return df

    def extract_from_api(self, url: str, schema: StructType = None):
      response = requests.get(url)
      rdd = spark.sparkContext.parallelize(response.json())
      if schema:
        df = spark.read.schema(schema).json(rdd)
      else:
        df = spark.read.json(rdd)
      return df

    def load(self, df: DataFrame, format: str, path: str, **kwargs) -> None:
        df.write.mode("overwrite").format(format).save(path)

class ETLTask(ETLFlow):

    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def ingestion_vehicles(self):
      vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                                  StructField('block_id', StringType(), True),
                                  StructField('current_status', StringType(), True),
                                  StructField('id', StringType(), True),
                                  StructField('lat', FloatType(), True),
                                  StructField('line_id', StringType(), True),
                                  StructField('lon', FloatType(), True),
                                  StructField('pattern_id', StringType(), True),
                                  StructField('route_id', StringType(), True),
                                  StructField('schedule_relationship', StringType(), True),
                                  StructField('shift_id', StringType(), True),
                                  StructField('speed', FloatType(), True),
                                  StructField('stop_id', StringType(), True),
                                  StructField('timestamp', TimestampType(), True),
                                  StructField('trip_id', StringType(), True)])

      df = self.extract_from_api(url="https://api.carrismetropolitana.pt/vehicles", schema=vehicle_schema)
      df = df.withColumn('date', date_format('timestamp',"HHmmss"))
      self.load(df=df, format="parquet", path="/content/lake/bronze/vehicles")

    def ingestion_lines(self):


      df = self.extract_from_api(url="https://api.carrismetropolitana.pt/lines")

      self.load(df=df, format="parquet", path="/content/lake/bronze/lines")

    def ingestion_municipalities(self):


      df = self.extract_from_api(url="https://api.carrismetropolitana.pt/municipalities")

      self.load(df=df, format="parquet", path="/content/lake/bronze/municipalities")

    def cleansing_vehicles(self):
      df = self.extract_from_file(format="parquet", path="/content/lake/bronze/vehicles")

      # transformations
      #df = df.withColumn("date", lit("test"))
      df = df.drop_duplicates()

      self.load(df=df, format="parquet", path="/content/lake/silver/vehicles")



    def enrich(self):
        pass


if __name__ == '__main__':

    # init spark
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.master('local').appName('ETL Program').getOrCreate()

    print("Starting ETL program")
    etl = ETLTask(spark)

    # run tasks
    print("Running Task - Ingestion Vehicles")
    etl.ingestion_vehicles()

    print("Running Task - Ingestion lines")
    etl.ingestion_lines()

    print("Running Task - ingestion municipalities ")
    etl.ingestion_municipalities()

    print("Running Task - Cleansing Vehicles")
    etl.cleansing_vehicles()

    #etl.enrich()

    print("ETL program completed")

Starting ETL program
Running Task - Ingestion Vehicles
Running Task - Ingestion lines
Running Task - ingestion municipalities 
Running Task - Cleansing Vehicles
ETL program completed


In [22]:

spark.read.parquet("/content/lake/bronze/vehicles").show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|  date|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+------+
|    198|20241122-64010340...| IN_TRANSIT_TO|44|12063|  38.5313|   4426|-8.885352|  4426_0_1|  4426_0|            SCHEDULED|112340234560|      0.0| 160266|2024-11-22 16:45:52|4426_0_1|2200|170...|164552|
|    142|20241122-64010062...| IN_TRANSIT_TO|44|12506|38.617672|   4621|-9.102014|  4621_0_2|  4621_0|            SCHEDULED|121810234560|      0.0| 140045|2024-11-22 16:46:54|4621_0_2|

756
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|    271|20241122-64010230...| IN_TRANSIT_TO|44|12074|38.522186|   4426|-8.861442|  4426_0_2|  4426_0|            SCHEDULED|112210234560|      0.0| 160237|2024-11-22 16:05:09|4426_0_2|2200|161...|
|     67|20241122-64010191...|   INCOMING_AT|44|12548| 38.52589|   4409|-8.910276|  4409_0_2|  4409_0|            SCHEDULED|112580234560|4.7222223| 160264|2024-11-22 16:05:07|4409_0_2|2200|160...|
|    331|20

In [30]:
spark.read.parquet("/content/lake/bronze/lines").show()

+--------------------+-------+----------+----+--------------------+--------------------+--------------+--------------------+--------------------+----------+----------+
|     _corrupt_record|  color|facilities|  id|          localities|           long_name|municipalities|            patterns|              routes|short_name|text_color|
+--------------------+-------+----------+----+--------------------+--------------------+--------------+--------------------+--------------------+----------+----------+
|                NULL|#C61D23|        []|1001|[Alfragide, Amado...|Alfragide (Estr S...|        [1115]|[1001_0_1, 1001_0_2]|            [1001_0]|      1001|   #FFFFFF|
|                NULL|#C61D23|        []|1002|[Reboleira, Amado...|Reboleira (Estaçã...|        [1115]|          [1002_0_3]|            [1002_0]|      1002|   #FFFFFF|
|                NULL|#C61D23|        []|1003|[Amadora, Amadora...|Amadora (Estação ...|        [1115]|[1003_0_1, 1003_0_2]|            [1003_0]|      1003|   #